In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('OnlineRetail.xlsx')

In [ ]:
df

In [ ]:
df.isnull().sum()

# Question 1

 Find the top 10 customers who spent the most money Using MapReduce.

In [ ]:
#cleaning data for this part
df2=df.dropna(subset=['CustomerID'])
df2

In [ ]:
#As we know, we shouldn't calculate the number of those who had cancellation.
df2 = df2[~(df['InvoiceNo'].astype('string').str.contains('C'))]
df2

First, I choose 3 columns that are needed for answering this part.

In [ ]:
my_data= pd.concat([df2['CustomerID'],df2['Quantity'],df2['UnitPrice']], axis=1)

In [ ]:
my_data

Because of using Mapreduce, first, I split the table into 10 new tables.

In [ ]:
total_rows = len(my_data)

rows_per_table = total_rows // 10

tables = []
start_row = 0

for i in range(10):
    end_row = start_row + rows_per_table
    table = my_data[start_row:end_row]
    tables.append(table)
    start_row = end_row
#I stored my new tables in "tables".

#showing the tables:
for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    print(table)
    print()


In [ ]:
for tabl in tables:
    tabl['price'] = tabl['Quantity'] * tabl['UnitPrice']
    
for i, tabl in enumerate(tables):
    print(f"Table {i+1}:")
    print(tabl)
    print()

Creating (numbers, CustomerID, price)

In [ ]:
my_list=[]
for table in tables:
    result=[]
    numbers = list(range(1, len(table['CustomerID'])))
    result = list(zip(numbers,table['CustomerID'], table['price']))
    my_list.append(result)
    
for i, result in enumerate(my_list):
    print(f"Table {i+1}:")
    print(result)

giving 1 to each key in tables

In [ ]:
dictionaries = []

for result in my_list:
    my_dict = {}
    for key in result:
        my_dict[key] = [1]
    dictionaries.append(my_dict)

# Print the dictionaries
for i, my_dict in enumerate(dictionaries):
    print(f"Table {i+1}:")
    for key, values in my_dict.items():
        print(key, values)
    print()

collecting: (CustumerID, price): [1,...,1]

In [ ]:
lastlist = []
for dictionary in dictionaries:
    firstlist = {}
    for key in dictionary.keys():
        new_key = key[1:3]
        if new_key in firstlist:
            firstlist[new_key].append(1)
        else:
            firstlist[new_key] = [1]
    lastlist.append(firstlist)
    
for i, firstlist in enumerate(lastlist):
    print(f"Table {i+1}:")
    for key, values in firstlist.items():
        print(key, values)
    print()

collecting all the values from different tables

In [ ]:
def collect_values(dictionaries):
    common_values = {}
    
    for dictionary in dictionaries:
        for key in dictionary:
            if key in common_values:
                common_values[key].append(dictionary[key])
            else:
                common_values[key] = [dictionary[key]]
    
    return common_values


collect= collect_values(lastlist)
collect

Reducing

In [ ]:
def count_values(A):
    count_dict = {}
    for key, values in A.items():
        count = 0
        for sublist in values:
            count += sublist.count(1)
        count_dict[key] = count
    return count_dict

x = count_values(collect)
print(x)


Now, with a simple *, we can calculate how much money, each customer had spent

In [ ]:
def all_spent(mydict):
    new_dict = {}
    for key, value in mydict.items():
        CustomerID, price = key
        num = value
        spent= price*num
        if CustomerID in new_dict:
            new_dict[CustomerID].append(spent)
        else:
            new_dict[CustomerID] = [spent]
    
    return new_dict
myoutput = all_spent(x)
print(myoutput)

In [ ]:
sum_all = {key: sum(values) for key, values in myoutput.items()}
sum_all

In [ ]:
sorted_dict = {key: value for key, value in sorted(sum_all.items(), key=lambda item: item[1], reverse=True)}
my_tuple = tuple(sorted_dict.items())[:10]
print(my_tuple)


# Question 2

Calculate the total revenue generated by each customer for a given time period using MapReduce

In [ ]:
#We made this tables in question 1. Here, we only use it.
df3 = pd.concat([my_data, df2['InvoiceDate']], axis = 1)
df3

**Here, in 2 boxes below, we choose a period of time to calculate on it. After this part, other parts are like as "question 1".**

In [ ]:
from datetime import datetime
def filter_time(data, start_time, end_time):
    df = pd.DataFrame(data)
    df.set_index('InvoiceDate', inplace=True)
    filtered_df = df.loc[start_time:end_time]
    
    return filtered_df

    

In [ ]:
limited_tables = []
for table in tables:
    limited_table = filter_time(df3, datetime(2011,1,1) , datetime(2011,3,5))
    limited_tables.append(limited_table)
    
for i, limited_table in enumerate(limited_tables):
    print (f"Table {i+1}:")
    print(limited_table)
    

In [ ]:
for limited_tabl in limited_tables:
    limited_tabl['price'] = limited_tabl['Quantity'] * limited_tabl['UnitPrice']
    
for i, limited_table in enumerate(limited_tables):
    print(f"Table {i+1}:")
    print(limited_table)
    print()

In [ ]:
my_list=[]
for limited_table in limited_tables:
    result=[]
    numbers = list(range(1, len(limited_table['CustomerID'])))
    result = list(zip(numbers,limited_table['CustomerID'], limited_table['price']))
    my_list.append(result)
    
for i, result in enumerate(my_list):
    print(f"Table {i+1}:")
    print(result)

In [ ]:
dictionaries = []

for result in my_list:
    my_dict = {}
    for key in result:
        my_dict[key] = [1]
    dictionaries.append(my_dict)

# Print the dictionaries
for i, my_dict in enumerate(dictionaries):
    print(f"Table {i+1}:")
    for key, values in my_dict.items():
        print(key, values)
    print()

In [ ]:
lastlist = []
for dictionary in dictionaries:
    firstlist = {}
    for key in dictionary.keys():
        new_key = key[1:3]
        if new_key in firstlist:
            firstlist[new_key].append(1)
        else:
            firstlist[new_key] = [1]
    lastlist.append(firstlist)
    
for i, firstlist in enumerate(lastlist):
    print(f"Table {i+1}:")
    for key, values in firstlist.items():
        print(key, values)
    print()

In [ ]:
collect= collect_values(lastlist)
collect

In [ ]:
def count_values(A):
    count_dict = {}
    for key, values in A.items():
        count = 0
        for sublist in values:
            count += sublist.count(1)
        count_dict[key] = count
    return count_dict

y = count_values(collect)
print(y)


In [ ]:
my_output = all_spent(y)
print(my_output)

In [ ]:
sum_all = {key: sum(values) for key, values in my_output.items()}
sum_all

In [ ]:
sorted_dict = {key: value for key, value in sorted(sum_all2.items(), key=lambda item: item[1], reverse=True)}
my_tuple = tuple(sorted_dict.items())[:10]
print(my_tuple)


**Finished!**
So, you can see the total revenue generated by each customer for a period of time.